---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [9]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

FileNotFoundError: [Errno 2] No such file or directory: 'A4_graphs'

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
# import matplotlib.pyplot as plt
# %matploblib notebook

# for G in P1_Graphs:
#     degrees = G.degree()
#     degree_values = sorted(set(degrees.values()))
#     histogram = [list(degrees.values()).count(i)/float(nx.number_of_nodes(G)) for i in degree_values]
#     plt.figure()
#     plt.bar(degree_values,histogram) 
#     plt.xlabel('Degree') 
#     plt.ylabel('Fraction of Nodes') 
#     plt.show()
#     plt.close()

# for G in P1_Graphs:
#     print(nx.average_clustering(G), nx.average_shortest_path_length(G))

def graph_identification():
    
    
    return ['PA','SW_L','SW_L','PA','SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [2]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))


Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, roc_curve



In [4]:
df = pd.DataFrame(G.nodes())
df['salary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
df['clustering'] = pd.Series(nx.clustering(G))
df['degree'] = pd.Series(G.degree())
df['betweenness'] = pd.Series(nx.betweenness_centrality(G, normalized = True, endpoints = False))
df['closeness']=pd.Series(nx.closeness_centrality(G, normalized=True))


In [5]:
df['salary'].value_counts()

0.0    634
1.0    119
Name: salary, dtype: int64

In [6]:
df['salary']

0       0.0
1       NaN
2       NaN
3       1.0
4       1.0
5       NaN
6       1.0
7       0.0
8       NaN
9       0.0
10      0.0
11      0.0
12      1.0
13      1.0
14      NaN
15      0.0
16      0.0
17      0.0
18      NaN
19      0.0
20      0.0
21      1.0
22      0.0
23      0.0
24      0.0
25      0.0
26      0.0
27      NaN
28      1.0
29      1.0
       ... 
975     0.0
976     0.0
977     0.0
978     0.0
979     0.0
980     0.0
981     0.0
982     0.0
983     0.0
984     NaN
985     0.0
986     0.0
987     NaN
988     0.0
989     NaN
990     0.0
991     NaN
992     NaN
993     0.0
994     NaN
995     0.0
996     NaN
997     0.0
998     0.0
999     0.0
1000    NaN
1001    NaN
1002    0.0
1003    0.0
1004    0.0
Name: salary, Length: 1005, dtype: float64

In [6]:
def salary_predictions():
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.svm import SVC
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import auc, roc_curve
    df = pd.DataFrame(G.nodes())
    df['salary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['clustering'] = pd.Series(nx.clustering(G))
    df['degree'] = pd.Series(G.degree())
    df['betweenness'] = pd.Series(nx.betweenness_centrality(G, normalized = True, endpoints = False))
    df['closeness']=pd.Series(nx.closeness_centrality(G, normalized=True))
    df = df.set_index([0])
    df_test = df[df.salary.isnull()]
    df = df.dropna()
    feature=['clustering', 'degree', 'betweenness','closeness']
    X_train, X_test, y_train, y_test = train_test_split(df[feature], df['salary'])
    clf = RandomForestClassifier(max_depth = 4).fit(X_train, y_train)
    fpr, tpr,_ = roc_curve(y_test,clf.predict(X_test))
    print('RandomForest auc', auc(fpr,tpr))
    clf = SVC(kernel='poly').fit(X_train, y_train)
    fpr, tpr,_ = roc_curve(y_test,clf.predict(X_test))
    print('SVC auc', auc(fpr,tpr))
    y_pred = clf.predict_proba(df_test[feature])[:,1]
    df_test['salary'] = y_pred
    return df_test['salary']
salary_predictions()

RandomForest auc 0.763095799906
SVC auc 0.733246814535


AttributeError: predict_proba is not available when  probability=False

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [3]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [ ]:
def new_connections_predictions():
    df2 = future_connections
    df2['PreferentialAttachment'] = [i[2] for i in nx.preferential_attachment(G, df2.index)]
    df2['CommonNeighbors'] = df2.index.map(lambda node: len(list(nx.common_neighbors(G, node[0], node[1]))))
    df2['jaccard'] = [i[2] for i in nx.jaccard_coefficient(G, df2.index)]
    df2['resource_allocation'] = [i[2] for i in nx.resource_allocation_index(G, df2.index)]
    df_test = df2[df2['Future Connection'].isnull()]
    df2 = df2.dropna()
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import auc, roc_curve
    
    feature=['PreferentialAttachment','CommonNeighbors','jaccard', 'resource_allocation']
    X_train, X_test, y_train, y_test = train_test_split(df2[feature], df2['Future Connection'])
    clf = RandomForestClassifier(max_depth = 7).fit(X_train, y_train)
    df_test['pred'] = clf.predict_proba(df_test[feature])[:,1]
    return df_test['pred']
new_connections_predictions()